### Bind runtime args

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [2]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Write out the following equation using algebraic symbols then solve it.

            Use the format:
            
            EQUATION:...
            SOLUTION:...
            """,
        ),
        ("human", "{equation_statement}"),
    ]
)
model = ChatOpenAI(temperature=0)
runnable = (
    {"equation_statement": RunnablePassthrough()} | prompt | model | StrOutputParser()
)

print(runnable.invoke("x raised to the third plus seven equals 12"))

EQUATION: x^3 + 7 = 12

SOLUTION:
To solve this equation, we need to isolate the variable x.

Subtracting 7 from both sides of the equation, we get:

x^3 = 12 - 7
x^3 = 5

To solve for x, we need to take the cube root of both sides of the equation:

∛(x^3) = ∛5
x = ∛5

Therefore, the solution to the equation x^3 + 7 = 12 is x = ∛5.


In [3]:
runnable = (
    {"equation_statement": RunnablePassthrough()}
    | prompt
    | model.bind(stop="SOLUTION")
    | StrOutputParser()
)
print(runnable.invoke("x raised to the third plus seven equals 12"))

EQUATION: x^3 + 7 = 12




### Attaching OpenAI tools

In [4]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

In [5]:
model = ChatOpenAI().bind(tools=tools)
model.invoke("Whats the weather in SF, NYC and LA?")

Retrying langchain_community.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


AIMessage(content='', additional_kwargs={'tool_calls': [<OpenAIObject id=call_kVA2EfYE0MROdp8GFhzOeC6C at 0x12fb1c2c0> JSON: {
  "function": {
    "arguments": "{\n  \"location\": \"San Francisco, CA\"\n}",
    "name": "get_current_weather"
  },
  "id": "call_kVA2EfYE0MROdp8GFhzOeC6C",
  "type": "function"
}]})